In [1]:
import os
import pandas as pd
import re
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Downloading the required wordnet data
nltk.download('punkt')
from langdetect import detect

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\draxe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\draxe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\draxe\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\draxe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])
def tokenize_text(text):
    return word_tokenize(text)

In [3]:
def preprocess(df):
    # Convert 'combined_text' column to strings
    df['combined_text'] = df['combined_text'].astype(str)
    
    # Lowercase
    df['combined_text'] = df['combined_text'].str.lower()

    # Expanding Contractions
    df['combined_text'] = df['combined_text'].apply(lambda x: contractions.fix(x))

    # Removing the URLs
    df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x, flags=re.MULTILINE))

    # Removing the HTML tags
    df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(r'<.*?>', '', x))

    # Removing the numbers
    df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(r'\d+', '', x))

    # Removing the extra whitespaces
    df['combined_text'] = df['combined_text'].apply(lambda x: x.strip())

    # Removing the punctuation
    df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

    # Removing the stopwords
    df['combined_text'] = df['combined_text'].apply(remove_stopwords)

    # Tokenization
    df['combined_text'] = df['combined_text'].apply(tokenize_text)

    return df

# XG Boost

In [4]:
import os
import pandas as pd
import re
import contractions
from langdetect import detect

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [6]:
train = pd.read_csv('..\\..\\Dataset_Cleaned\\ISOT\\clean_train_isot.csv')

In [7]:
train = preprocess(train)

In [8]:
X_train = train['combined_text']
y_train = train['label']

In [9]:
X_train.head()

0    [conservative, terrorist, given, insanely, lig...
1    [clinton, ad, slam, trump, disgusting, insult,...
2    [hardliner, protest, french, labor, reform, ma...
3    [nba, kowtow, racist, order, player, stand, an...
4    [ben, carson, praise, time, trump, compared, c...
Name: combined_text, dtype: object

In [10]:
X_train = pd.Series(X_train)
X_train = X_train.astype(str)

In [13]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

# Define a pipeline combining a text feature extractor with a simple classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')),
])

# Define the grid parameters
params = [
    {
        'clf': [XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')],
        'clf__n_estimators': [100],  # number of gradient boosted trees
        'clf__learning_rate': [0.1],  # learning rate
    }
]

# Create a StratifiedKFold object
stratified_kfold = StratifiedKFold(n_splits=10)

# Create the grid search object
grid_search = GridSearchCV(pipeline, params, cv=stratified_kfold, n_jobs=-1, refit=True, verbose=2)

# Fit the grid search object to the data to compute the optimal model
grid_search.fit(X_train, y_train)

# Print the best score and the best parameters
print("Best Score: ", grid_search.best_score_)
print("Best Parameters: ", grid_search.best_params_)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best Score:  0.9961455023657624
Best Parameters:  {'clf': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...), 'clf__learning_rate': 0.1, 'clf__n_estimators': 100}


In [14]:
clf = grid_search.best_estimator_

In [15]:
# Training performance
y_train_pred = clf.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16933
           1       1.00      1.00      1.00     13940

    accuracy                           1.00     30873
   macro avg       1.00      1.00      1.00     30873
weighted avg       1.00      1.00      1.00     30873



# Submit test prediction

In [17]:
test = pd.read_csv('..\\..\\Dataset_Cleaned\\ISOT\\clean_test_isot.csv')

In [18]:
test = preprocess(test)

In [19]:
test = test.fillna(' ')
X_test = test['combined_text'].apply(' '.join)
test
y_test = test['label']
y_test_pred = clf.predict(X_test)

In [22]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      4258
           1       1.00      0.99      1.00      3461

    accuracy                           1.00      7719
   macro avg       1.00      1.00      1.00      7719
weighted avg       1.00      1.00      1.00      7719

